In [1]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
path = '/content/drive/MyDrive/Project_1/'
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith('.csv')]
file_list_csv

['power_user_curve.csv',
 '일간기준매출지표.csv',
 '월간기준매출지표.csv',
 '일별매출누계.csv',
 '월별_일별누계.csv',
 '7day_이동평균.csv',
 'ABC분석.csv',
 'products.csv',
 'pickup_shop.csv',
 'non_organic_install.csv',
 'daily_cost.csv',
 'daily_cost_202407041937.csv',
 'roas.csv',
 'nday_retention.csv',
 'organic_install.csv',
 '신규_재방문_복귀.csv',
 'daily_login_users_202407041937.csv',
 'myinfos.csv',
 'orders.csv',
 'marketing_myinfos_202407041938.csv',
 'login_users.csv',
 'roas_df.csv',
 'ltv.csv',
 'RoasByCampaign.csv',
 'monthly_roas_by_campaign.csv']

In [4]:
login = pd.read_csv(path + 'daily_login_users_202407041937.csv', encoding = 'utf-8')
orders = pd.read_csv(path + 'orders.csv', encoding = 'utf-8')
sales_index = pd.read_csv(path + '일간기준매출지표.csv', encoding = 'utf-8')

shop = pd.read_csv(path + 'pickup_shop.csv', encoding = 'utf-8')

In [5]:
orders['order_date'] = orders['order_date'].astype('datetime64[ns]')
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32224 entries, 0 to 32223
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          32224 non-null  object        
 1   order_date        32224 non-null  datetime64[ns]
 2   product_id        32224 non-null  object        
 3   user_id           32224 non-null  object        
 4   payment_amount    32224 non-null  int64         
 5   payment_status    32224 non-null  object        
 6   pickup_shop_name  32224 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.7+ MB


In [6]:
login['login_date'] = login['login_date'].astype('datetime64[ns]')
sales_index['login_date'] = sales_index['login_date'].astype('datetime64[ns]')

In [7]:
orders['order_year_month'] = orders['order_date'].dt.strftime('%Y-%m')

In [8]:
sales = orders.groupby(['pickup_shop_name'], as_index = False).agg({'payment_amount':'sum', 'user_id':'nunique'})
sales = sales.rename(columns = {'payment_amount':'sales', 'user_id':'pu'})
sales

,pickup_shop_name,sales,pu
0,픽업존(GS25 목동3동점),33700500,394
1,픽업존(GS영등포당산점),36577700,547
2,픽업존(다이소 난곡사거리점),12103000,170
3,픽업존(더현대서울),1199500,23
4,픽업존(메이비카페),76700200,653
5,픽업존(서울드림신용협동조합),37322600,419
6,픽업존(유승상가),267844000,1929
7,픽업존(참맛부대찌개아구찜),32495100,335


In [9]:
shop

,pickup_shop_name,region_1depth_name,region_2depth_name,region_3depth_name,h_code,x,y
0,픽업존(메이비카페),서울,구로구,구로동,1153054000,37.484929,126.900259
1,픽업존(다이소 난곡사거리점),서울,관악구,신림동,1162076500,37.481460,126.914207
2,픽업존(서울드림신용협동조합),서울,구로구,신도림동,1153051000,37.509568,126.882036
3,픽업존(참맛부대찌개아구찜),서울,영등포구,문래동3가,1156060500,37.516276,126.897390
4,픽업존(GS영등포당산점),서울,영등포구,당산동5가,1156056000,37.532379,126.896801
5,픽업존(유승상가),서울,양천구,목동,1147055000,37.539895,126.881942
6,픽업존(GS25 목동3동점),서울,양천구,목동,1147053000,37.546389,126.864435
7,픽업존(더현대서울),서울,영등포구,여의도동,1156054000,37.525191,126.929113


In [10]:
## 막대그래프
fig = px.bar(sales, x = 'pickup_shop_name', y = 'sales', color = 'pickup_shop_name', text = 'sales')
# << 지정된 카테고리 순서에 맞춰 정렬 >>
fig.update_xaxes(categoryorder='array', categoryarray= ['픽업존(다이소 난곡사거리점)','픽업존(유승상가)','픽업존(참맛부대찌개아구찜)','픽업존(GS25 목동3동점)','픽업존(더현대서울)','픽업존(GS영등포당산점)','픽업존(서울드림신용협동조합)','픽업존(메이비카페)'])

fig.show(renderer='colab')

In [11]:
##달 별 sales 누적 막대 그래프

#데이터 핸들링
sales_temp = orders.groupby(['order_year_month', 'pickup_shop_name'])['payment_amount'].sum().reset_index()
sales_temp = sales_temp.melt(id_vars = ['order_year_month', 'pickup_shop_name'], value_vars = ['payment_amount'])
sales_temp

#시각화
fig = px.bar(sales_temp, x = 'order_year_month', y = 'value', color = 'pickup_shop_name', text = 'value')
fig.show(renderer='colab')

In [12]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   pickup_shop_name  8 non-null      object
 1   sales             8 non-null      int64 
 2   pu                8 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 320.0+ bytes


## 마케팅 데이터 분석


### 비용

In [13]:
daily_cost = pd.read_csv(path + 'daily_cost.csv', encoding = 'utf-8')
daily_cost['aggregate_date'] = daily_cost['aggregate_date'].astype('datetime64[ns]')
daily_cost['aggregate_year_month'] = daily_cost['aggregate_date'].dt.strftime('%Y-%m')
daily_cost = daily_cost.groupby(['aggregate_date', 'aggregate_year_month'], as_index = False).agg({'cost':'sum'})
daily_cost

,aggregate_date,aggregate_year_month,cost
0,2022-05-01,2022-05,126032.3788
1,2022-05-02,2022-05,101023.8217
2,2022-05-03,2022-05,175948.3153
3,2022-05-04,2022-05,251600.9564
4,2022-05-05,2022-05,151227.3761
...,...,...,...
209,2022-11-26,2022-11,395067.6959
210,2022-11-27,2022-11,125510.0799
211,2022-11-28,2022-11,466152.0236
212,2022-11-29,2022-11,527889.3104


In [14]:
#일간 마케팅 비용 막대그래프

fig = px.bar(daily_cost,
             x ='aggregate_date',
             y = 'cost',
             color='aggregate_year_month',)

fig.show(renderer='colab')

In [15]:
## 월간 마케팅 비용

fig = px.bar(daily_cost, x = 'aggregate_year_month', y = 'cost')

fig.show()

### 모객 유저수

#### 일간 모객 유저 수

In [16]:
marketing_myinfos = pd.read_csv(path + 'marketing_myinfos_202407041938.csv', encoding = 'utf-8')
organic = pd.read_csv(path + 'organic_install.csv', encoding = 'utf-8')
non_organic = pd.read_csv(path + 'non_organic_install.csv', encoding = 'utf-8')
marketing_myinfos['create_date'] = marketing_myinfos['create_date'].astype('datetime64[ns]')

In [17]:
marketing_myinfos

,create_date,create_year_month,user_id,nickname,account_type,platform,region_1depth_name,region_2depth_name,region_3depth_name,h_code,x,y,attribute_id
0,2022-05-01,2022-05,7b182e45-c7ff-48c6-8533-c773b0c543da,박성수-7b182e45,naver,android,서울,양천구,목동,1147053000,37.5455,126.8655,attribute-7b182e45-c7ff-48c6-8533-c773b0c543da
1,2022-05-01,2022-05,de7cc8c3-05c6-418c-ae70-ab4516ec0767,이정숙-de7cc8c3,kakao,android,서울,양천구,목동,1147055000,37.5416,126.8826,attribute-de7cc8c3-05c6-418c-ae70-ab4516ec0767
2,2022-05-01,2022-05,3ef0ce1a-c463-47ef-9c9a-930bf2f268e3,민현준-3ef0ce1a,naver,android,서울,양천구,목동,1147055000,37.5416,126.8826,attribute-3ef0ce1a-c463-47ef-9c9a-930bf2f268e3
3,2022-05-01,2022-05,4ac3e42d-4ee0-446b-8829-093bf59b6e36,윤서준-4ac3e42d,kakao,android,서울,영등포구,문래동3가,1156060500,37.5161,126.8967,attribute-4ac3e42d-4ee0-446b-8829-093bf59b6e36
4,2022-05-01,2022-05,aab15bb1-e6d9-4ca2-88a3-2a2659626ce3,윤정수-aab15bb1,kakao,android,서울,양천구,목동,1147055000,37.5370,126.8764,attribute-aab15bb1-e6d9-4ca2-88a3-2a2659626ce3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7704,2022-11-30,2022-11,b4acd4ca-494e-4a0d-8e79-e56c39053868,허재현-b4acd4ca,naver,android,서울,구로구,구로2동,1153053000,37.4921,126.8877,attribute-b4acd4ca-494e-4a0d-8e79-e56c39053868
7705,2022-11-30,2022-11,45b8a4a6-e143-4a72-a036-f453d62ab810,심숙자-45b8a4a6,naver,ios,서울,양천구,목동,1147055000,37.5416,126.8826,attribute-45b8a4a6-e143-4a72-a036-f453d62ab810
7706,2022-11-30,2022-11,c84a3490-7587-4f44-9553-f76d487e7abd,윤수진-c84a3490,naver,android,서울,영등포구,당산동5가,1156056000,37.5329,126.9028,attribute-c84a3490-7587-4f44-9553-f76d487e7abd
7707,2022-11-30,2022-11,e4d34b43-fbc9-42e8-9043-8fd98cdee620,권은서-e4d34b43,kakao,android,서울,양천구,목동,1147055000,37.5412,126.8785,attribute-e4d34b43-fbc9-42e8-9043-8fd98cdee620


In [18]:
organic['install_time'] = organic['install_time'].astype('datetime64[ns]')
non_organic['install_time'] = non_organic['install_time'].astype('datetime64[ns]')

In [19]:
organic['attribute_type'] = 'organic'
non_organic['attribute_type'] = 'non_organic'

b = pd.concat([organic, non_organic])
b = b[['attribute_type', 'install_time', 'attribute_id', 'campaign']]

user_info = pd.merge(
    marketing_myinfos,
    b,
    left_on=['attribute_id', 'create_date'],
    right_on=['attribute_id', 'install_time'],
    how='left'
)
user_info = user_info.groupby(['create_date', 'attribute_type'])['user_id'].nunique().reset_index()
user_info = user_info.rename(columns={'user_id': 'user_cnt'})

user_info = user_info.sort_values(by=['create_date', 'attribute_type'])
user_info

,create_date,attribute_type,user_cnt
0,2022-05-01,non_organic,5
1,2022-05-01,organic,4
2,2022-05-02,non_organic,4
3,2022-05-02,organic,5
4,2022-05-03,non_organic,7
...,...,...,...
423,2022-11-28,organic,16
424,2022-11-29,non_organic,15
425,2022-11-29,organic,38
426,2022-11-30,non_organic,14


In [20]:
## 시각화 - Attribute 타입별 일간 가입 유저수

import plotly.express as px

fig = px.bar(user_info,
             x ='create_date',
             y = 'user_cnt',
             color='attribute_type',)

fig.show(renderer='colab') #organic user가 더 많은것으로 보임

#### 월간 모객 유저수

In [21]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7709 entries, 0 to 2452
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   attribute_type  7709 non-null   object        
 1   install_time    7709 non-null   datetime64[ns]
 2   attribute_id    7709 non-null   object        
 3   campaign        2453 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 301.1+ KB


In [22]:
marketing_myinfos['create_year_month'] = marketing_myinfos['create_year_month'].astype('datetime64[ns]')

In [23]:
user_info = pd.merge(
    marketing_myinfos,
    b,
    left_on=['attribute_id', 'create_date'],
    right_on=['attribute_id', 'install_time'],
    how='left'
)

user_info = user_info.groupby(['create_year_month', 'attribute_type'])['user_id'].nunique().reset_index()
user_info = user_info.rename(columns={'user_id': 'user_cnt'})

user_info = user_info.sort_values(by=['create_year_month', 'attribute_type'])
user_info

,create_year_month,attribute_type,user_cnt
0,2022-05-01,non_organic,192
1,2022-05-01,organic,320
2,2022-06-01,non_organic,432
3,2022-06-01,organic,645
4,2022-07-01,non_organic,340
5,2022-07-01,organic,739
6,2022-08-01,non_organic,212
7,2022-08-01,organic,492
8,2022-09-01,non_organic,176
9,2022-09-01,organic,443


In [24]:
## << 시각화 - Attribute 타입별 월간 가입 유저수  >>

import plotly.express as px

fig = px.bar(user_info,
             x ='create_year_month',
             y = 'user_cnt',
             color= 'attribute_type',)

fig.show(renderer='colab')

#### 데이터프레임 생성 - 유저그룹(all/organic/non_organic), 월간 가입유저수, SALES, LTV


### 마케팅 진행 상황

- 현재 로컬팜은 단일 매체(페이스북)를 통한 마케팅 진행중 입니다.
- 로컬팜 서비스 초기에는 서울 지역 전역을 대상으로 한 `SEOUL` 캠페인만 진행하였으나. 현재는 예산을 나누어서 오프라인 매장이 위치한 동 단위 캠페인(`LC_Mok-dong`, `LC_Guro-dong`, `LC_Mullae-dong`,  `LC_Sindorim-dong`, `LC_Sillim-dong`, `LC_Dangsan-dong`)도 추가로 진행하고 있습니다.
- iOS, Android모두 Attribute 데이터가 정상적으로 수집되고 있음을 가정합니다.

### 회사 상황 및 요구사항

- 현재 로컬팜에는 전문 마케터가 없는 상태로 사업 담당자가 마케팅을 배워가며 진행하고 있습니다.
- 담당자는 본업으로 인해 캠페인을 설정하고, 비용을 소진하기에도 바쁘고 제대로 된 모객 효율 평가는 진행하지 못하고 있는 상황 입니다.
- 로컬팜 사업부에서는 데이터분석가에게 지금까지 수집된 데이터들을 토대로 모객 효율 평가 및 가이드라인 제안을 요청하였습니다.

### 모객효율 평가 지표
 1) LTV(Life Time Value) - 특정 코호트 집단(특정 캠페인으로 모객된 고객 그룹)에 대해 **시간의 흐름에 따른 인당 평균 구매액을 측정한 지표**, 코호트 그룹 간 매출 비교를 위해 사용 -> 특정 코호트 집단에 대해 동일 기간동안 추적하여 측정

 2) ROAS(Return On Ad Spend) - 마케팅 비용 회수 시점을 계산하기 위한 지표,
   LTV / 비용
   100% 달성 시점이 빠를수록, 이후 추가 매출이 지속적으로 발생할수록 코호트 집단에 대한 모객 효율이 높다고 판단

In [25]:
## sql에서 작업해놓은 ltv.csv 불러오기

ltv = pd.read_csv(path + 'ltv.csv', encoding = 'utf-8')
ltv.head()

,ltv_period,create_year_month,attribute_type,user_cnt,sales,ltv
0,D15,2022-05,all,512,1476900.0,2885.0
1,D15,2022-05,non_organic,192,495700.0,2582.0
2,D15,2022-05,organic,320,981200.0,3066.0
3,D15,2022-06,all,1077,3284600.0,3050.0
4,D15,2022-06,non_organic,432,1400300.0,3241.0


In [26]:
#월간 가입 유저 그룹별 LTV

## attribute_type별 ltv 나누기
all_ltv = ltv[ltv['attribute_type'] == 'all']
organic_ltv = ltv[ltv['attribute_type'] == 'organic']
non_organic_ltv = ltv[ltv['attribute_type'] == 'non_organic']

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols = 1)

fig.add_trace(go.Bar(x = [all_ltv['create_year_month'].tolist(), all_ltv['ltv_period'].tolist()], y = all_ltv['ltv'].tolist(), name = 'all'), row = 1, col = 1)
fig.add_trace(go.Bar(x = [organic_ltv['create_year_month'].tolist(), organic_ltv['ltv_period'].tolist()], y= organic_ltv['ltv'].tolist(), name = "organic"), row=2, col=1)
fig.add_trace(go.Bar(x = [non_organic_ltv['create_year_month'].tolist(), non_organic_ltv['ltv_period'].tolist()], y= non_organic_ltv['ltv'].tolist(), name = "non-organic"), row=3, col=1)

fig.show(renderer='colab')

In [27]:
### SQL에서 작업한 roasByCampaign.csv가져오기
roas = pd.read_csv(path + 'RoasByCampaign.csv', encoding = 'utf-8')
roas['attribute_type'].value_counts()

attribute_type
all            42
non_organic    42
organic        42
Name: count, dtype: int64

In [28]:
## 11월은 지표추적기간 완료X 이므로 데이터에서 예외처리
all_roas = roas[(roas['attribute_type']=='all') & (roas['create_year_month'] != '2022-11')]
organic_roas = roas[(roas['attribute_type']=='organic') & (roas['create_year_month'] != '2022-11')]
non_organic_roas = roas[(roas['attribute_type']=='non_organic') & (roas['create_year_month'] != '2022-11')]

#subplot 골격 설정, (y축 이중축 허용, 그래프 간격 설정)
fig = make_subplots(rows = 3, cols = 1, specs=[[{'secondary_y':True}], [{'secondary_y':True}],[{'secondary_y':True}]],
                    vertical_spacing=0.15)

#LTV 막대그래프 그리기
fig.add_trace(go.Bar(x = [all_roas['create_year_month'].tolist(), all_roas['ltv_period'].tolist()],
                     y = all_roas['ltv'].tolist(), name='LTV(all)'),
              secondary_y=False,
              row=1, col=1)
fig.add_trace(go.Bar(x = [organic_roas['create_year_month'].tolist(), organic_roas['ltv_period'].tolist()],
                     y = organic_roas['ltv'].tolist(), name='LTV(organic)'),
              secondary_y=False,
              row=2, col=1)
fig.add_trace(go.Bar(x = [non_organic_roas['create_year_month'].tolist(), non_organic_roas['ltv_period'].tolist()],
                     y = non_organic_roas['ltv'].tolist(), name='LTV(non_organic)'),
              secondary_y=False,
              row=3, col=1)

#Roas 라인 + 마커 그래프 그리기
fig.add_trace(
    go.Scatter(
        x = [all_roas['create_year_month'].tolist(), all_roas['ltv_period'].tolist()],
        y = all_roas['roas'].tolist(),
        name = 'Roas(all)',
        mode = 'lines+markers+text',
        text = all_roas['roas'],
        textposition = 'top center',
        texttemplate = '%{text:.0%}' # %{변수:변수포맷}의 형태로 사용
    ),
    secondary_y = True,
    row=1, col = 1
)

fig.add_trace(
    go.Scatter(
        x = [organic_roas['create_year_month'].tolist(), organic_roas['ltv_period'].tolist()],
        y = organic_roas['roas'].tolist(),
        name = 'Roas(organic)',
        mode = 'lines+markers+text',
        text = organic_roas['roas'],
        textposition = 'top center',
        texttemplate = '%{text:.0%}' # %{변수:변수포맷}의 형태로 사용
    ),
    secondary_y = True,
    row=2, col = 1
)

fig.add_trace(
    go.Scatter(
        x = [non_organic_roas['create_year_month'].tolist(), non_organic_roas['ltv_period'].tolist()],
        y = non_organic_roas['roas'].tolist(),
        name = 'Roas(non_organic)',
        mode = 'lines+markers+text',
        text = non_organic_roas['roas'],
        textposition = 'top center',
        texttemplate = '%{text:.0%}' # %{변수:변수포맷}의 형태로 사용
    ),
    secondary_y = True,
    row=3, col = 1
)

# 출력되는 그래프 레이아웃 상세 조정
fig.update_layout(height = 900) # e.g.(height=600, width=600)
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 가입유저 LTV, ROAS ><b>",
            "x": 0.5, # An int or float in the interval [0, 1]
            "y": 0.95, # An int or float in the interval [0, 1]
            "font": {
                "size": 13
            }
        },
        "yaxis1": {
            "title": "LTV (원)"
        },
        "yaxis2": {
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
        "yaxis3": {
            "title": "LTV (원)"
        },
        "yaxis4": {
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
        "yaxis5": {
            "title": "LTV (원)"
        },
        "yaxis6": {
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },

    },
)

# 각 서브플랏에서, 이중축으로 잡혀있는 ROAS 그리드 지우기
fig['layout']['yaxis2']['showgrid'] = False
fig['layout']['yaxis4']['showgrid'] = False
fig['layout']['yaxis6']['showgrid'] = False

# 그래프 출력
fig.show(renderer='colab')

In [29]:
RoasByCampaign = pd.read_csv('/content/drive/MyDrive/Project_1/monthly_roas_by_campaign.csv', encoding = 'utf-8')
RoasByCampaign

,ltv_period,create_year_month,attribute_type,campaign,user_cnt,sales,ltv,cost,roas
0,D15,2022-05,non_organic,SEOUL,192,495700.0,2582.0,4.850834e+06,0.102
1,D30,2022-05,non_organic,SEOUL,192,1283300.0,6684.0,4.850834e+06,0.265
2,D45,2022-05,non_organic,SEOUL,192,1702100.0,8865.0,4.850834e+06,0.351
3,D60,2022-05,non_organic,SEOUL,192,2299600.0,11977.0,4.850834e+06,0.474
4,D75,2022-05,non_organic,SEOUL,192,2806200.0,14616.0,4.850834e+06,0.578
...,...,...,...,...,...,...,...,...,...
235,D30,2022-11,organic,NaN,2046,NaN,NaN,NaN,NaN
236,D45,2022-11,organic,NaN,2046,NaN,NaN,NaN,NaN
237,D60,2022-11,organic,NaN,2046,NaN,NaN,NaN,NaN
238,D75,2022-11,organic,NaN,2046,NaN,NaN,NaN,NaN


In [34]:
## 시각화

#11월은 지표 추적기간이 다 차지 않았으므로 데이터에서 예외처리
seoul_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'SEOUL') & (RoasByCampaign['create_year_month']!='2022-11')]
mokdong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Mok-dong') & (RoasByCampaign['create_year_month']!='2022-11')]
gurodong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Guro-dong') & (RoasByCampaign['create_year_month']!='2022-11')]
mullaedong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Mullae-dong') & (RoasByCampaign['create_year_month']!='2022-11')]
sillimdong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Sillim-dong') & (RoasByCampaign['create_year_month']!='2022-11')]
dangsandong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Dangsan-dong') & (RoasByCampaign['create_year_month']!='2022-11')]
sindorimdong_roas = RoasByCampaign[(RoasByCampaign['campaign'] == 'LC_Sindorim-dong') & (RoasByCampaign['create_year_month']!='2022-11')]

# subplot 골격 설정(2x1, 그래프 간격 그리기)
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.15)

#캠페인별 가입 유저수 막대그래프 그리기
fig.add_trace(go.Bar(x = [seoul_roas['create_year_month'].tolist(), seoul_roas['ltv_period'].tolist()],
                     y = seoul_roas['user_cnt'].tolist(), name = '유저수(SEOUL)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [mokdong_roas['create_year_month'].tolist(), mokdong_roas['ltv_period'].tolist()],
                     y = mokdong_roas['user_cnt'].tolist(), name = '유저수(mokdong)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [gurodong_roas['create_year_month'].tolist(), gurodong_roas['ltv_period'].tolist()],
                     y = gurodong_roas['user_cnt'].tolist(), name = '유저수(gurodong)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [mullaedong_roas['create_year_month'].tolist(), mullaedong_roas['ltv_period'].tolist()],
                     y = mullaedong_roas['user_cnt'].tolist(), name = '유저수(mullaedong)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [sillimdong_roas['create_year_month'].tolist(), sillimdong_roas['ltv_period'].tolist()],
                     y = sillimdong_roas['user_cnt'].tolist(), name = '유저수(sillimdong)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [dangsandong_roas['create_year_month'].tolist(), dangsandong_roas['ltv_period'].tolist()],
                     y = dangsandong_roas['user_cnt'].tolist(), name = '유저수(dangsandong)'),
              row = 1, col = 1)
fig.add_trace(go.Bar(x = [sindorimdong_roas['create_year_month'].tolist(), sindorimdong_roas['ltv_period'].tolist()],
                     y = sindorimdong_roas['user_cnt'].tolist(), name = '유저수(sindorimdong)'),
              row = 1, col = 1)

# 캠페인별 ROAS 막대그래프 그리기
fig.add_trace(go.Bar(x = [seoul_roas['create_year_month'].tolist(), seoul_roas['ltv_period'].tolist()],
                     y = seoul_roas['roas'].tolist(), name = 'ROAS(SEOUL)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [mokdong_roas['create_year_month'].tolist(), mokdong_roas['ltv_period'].tolist()],
                     y = mokdong_roas['roas'].tolist(), name = 'ROAS(mokdong)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [gurodong_roas['create_year_month'].tolist(), gurodong_roas['ltv_period'].tolist()],
                     y = gurodong_roas['roas'].tolist(), name = 'ROAS(gurodong)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [mullaedong_roas['create_year_month'].tolist(), mullaedong_roas['ltv_period'].tolist()],
                     y = mullaedong_roas['roas'].tolist(), name = 'ROAS(mullaedong)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [sillimdong_roas['create_year_month'].tolist(), sillimdong_roas['ltv_period'].tolist()],
                     y = sillimdong_roas['roas'].tolist(), name = 'ROAS(sillimdong)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [dangsandong_roas['create_year_month'].tolist(), dangsandong_roas['ltv_period'].tolist()],
                     y = dangsandong_roas['roas'].tolist(), name = 'ROAS(dangsandong)'),
              row = 2, col = 1)
fig.add_trace(go.Bar(x = [sindorimdong_roas['create_year_month'].tolist(), sindorimdong_roas['ltv_period'].tolist()],
                     y = sindorimdong_roas['roas'].tolist(), name = 'ROAS(sindorimdong)'),
              row = 2, col = 1)

# 출력되는 그래프 레이아웃 상세 조정
fig.update_layout(height = 900) # e.g.(height=600, width=600)
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 가입유저 LTV, ROAS ><b>",
            "x": 0.5, # An int or float in the interval [0, 1]
            "y": 0.95, # An int or float in the interval [0, 1]
            "font": {
                "size": 13
            }
        },
        "yaxis": {
            "title": "가입유저수(명)",
        },
        "yaxis2": {
            "title": "ROAS",
            "tickformat": '.0%',
            "range": [0, 3],
        },
    },
)

# 그래프 출력
fig.show(renderer='colab')

## **[사용자 유입 분석]**

### <<MAU, NRU>>

In [38]:
#SQL로 처리한 csv파일 불러오기
mau = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/mau.csv', encoding = 'utf-8')
nru = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/nru.csv', encoding = 'utf-8')

In [39]:

# 시각화
## 강조하고 싶은 부분 crimson 색으로 표시
colors = ['lightgrey',]* len(mau)
colors[len(mau) - 1] = 'crimson'

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = mau['login_year_month'],
        y = mau['login_user_cnt'],
        name = 'MAU(월간 활성 유저수)',
        text = mau['login_user_cnt'],
        textposition = 'auto',
        texttemplate = '%{text:,}명',
        marker_color = colors
    )
)

fig.add_trace(
    go.Scatter(
        x = nru[nru['create_year_month'] >= '2022-01']['create_year_month'],
        y = nru[nru['create_year_month'] >= '2022-01']['user_cnt'],
        name = 'NRU(신규 유저수)',
        mode = 'lines+markers+text',
        text = nru[nru['create_year_month']>= '2022-01']['user_cnt'],
        textposition = 'top center',
        texttemplate = '%{text:,}명',
    )
)
## layout 설정
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 MAU, NRU ><b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 13
            }
        },
        "yaxis": {
            "title": "MAU, NRU(명)", "tickformat": ','
        },
        "xaxis": {
            "tick0": "2022-01",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "template":'plotly_white',
        "legend": {'orientation': 'h', 'x': 0, 'y': 1.2}
    },
)
## fig의 y축 기준 직사각형 영역표시 Or 텍스트 남김
fig.add_vrect(x0="2021-12-15",
              x1="2022-04-15",
              annotation_text="* 활성 유저수 지표는 22년 5월부터 확인 가능",
              annotation_position="top left",
              fillcolor="black", opacity=0.1, line_width=0)

fig.add_vrect(x0="2022-07-18",
              x1="2022-10-15",
              annotation_text="* MAU 점진적 하락 구간",
              annotation_position="top left",
              fillcolor="hotpink", opacity=0.1, line_width=0)

fig.show(renderer='colab')

## <<지역별 MAU, NRU>>

#### <매장별 거주 지역별 11월 매출>

In [44]:
#SQL로 처리한 csv 데이터 불러오기 및 핸들링
novShopAndPuRegion = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/NovShopAndPuRegion.csv', encoding = 'utf-8')
novShopAndPuRegion = novShopAndPuRegion.fillna(0) #NaN값 0으로 대체

In [45]:
novShopAndPuRegion

,order_year_month,pickup_shop_name,Sillim_dong,Jowon_dong,Guro2_dong,Sindorim_dong,Doksan3_dong,Mok_dong,Dangsandong4_ga,Dangsandong5_ga,Daerim2_dong,Mullaedong3_ga,Yangpyeongdong3_ga,Yangpyeongdong4_ga,Yeouido_dong
0,2022-11,픽업존(유승상가),0.0,0.0,0.0,0.0,0.0,51315800.0,0.0,0.0,0.0,2748000.0,0.0,0.0,0.0
1,2022-11,픽업존(메이비카페),0.0,0.0,2149800.0,0.0,837700.0,9565500.0,0.0,0.0,11978700.0,325300.0,0.0,0.0,0.0
2,2022-11,픽업존(서울드림신용협동조합),0.0,0.0,0.0,9418500.0,0.0,0.0,0.0,0.0,0.0,76100.0,0.0,0.0,0.0
3,2022-11,픽업존(GS25 목동3동점),0.0,0.0,0.0,179400.0,0.0,8660200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-11,픽업존(참맛부대찌개아구찜),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6137600.0,0.0,0.0,0.0
5,2022-11,픽업존(GS영등포당산점),0.0,0.0,0.0,0.0,72000.0,813400.0,10493300.0,21598900.0,0.0,0.0,2232800.0,1193600.0,173700.0
6,2022-11,픽업존(다이소 난곡사거리점),10452400.0,0.0,0.0,0.0,1477100.0,0.0,0.0,0.0,88600.0,0.0,0.0,0.0,84900.0
7,2022-11,픽업존(더현대서울),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1199500.0


In [46]:
## 월간 지역별 활성 유저수
mauByRegion = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/MauByRegion.csv', encoding = 'utf-8')

In [49]:
## <지역별 MAU Bar Chart로 그리기
fig = px.bar(mauByRegion, x = 'login_year_month', y = 'user_cnt', color = 'region', text = 'user_cnt')

fig.update_traces(texttemplate = '%{text:,}명', textposition = 'inside')

fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 지역별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  },
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "거주지역",
        "template":'plotly_white',
    }
)
fig.add_vrect(x0="2022-10-15",
              x1="2022-11-17",
              annotation_text="* 11월 신규매장 오픈 이후, 기존 지역의 AU 볼륨 전월比 상승 (문래동3가 제외)",
              annotation_position="top right",
              fillcolor="skyblue", opacity=0.25, line_width=0)

fig.show()

#### <월간 지역별 가입 유저수>

In [51]:
## SQL로 처리한 csv 불러오기 및 데이터 전처리
monthlyNruByRegion = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/monthlyNruByRegion.csv', encoding = 'utf-8')

## < monthlyAuAndNru: 월간 지역별 활성 유저수, 가입 유저수 >

mauByRegion = mauByRegion.rename(columns={'login_year_month':'year_month'})
mauByRegion['category'] = 'AU'
monthlyNruByRegion = monthlyNruByRegion.rename(columns={'create_year_month':'year_month'})
monthlyNruByRegion['category'] = 'NRU'
monthlyAuAndNru = pd.concat([mauByRegion, monthlyNruByRegion], ignore_index=True)

# user_group 한글로 변경
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'new', '신규지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'decrease', '감소지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_0_10', '10%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_10_20', '10%초과_20%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_20_30', '20%초과_30%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_30_more', '30%_초과_증가지역', monthlyAuAndNru['user_group'])

In [53]:
## <지역별 AU, NRU 시각화>

fig = px.line(monthlyAuAndNru,
              x="year_month",
              y= "user_cnt",
              color="region",
              facet_col = "user_group",
              facet_row = "category",
              markers=True,
              category_orders={"user_group": ["신규지역", "감소지역", "10%이하_증가지역", "30%_초과_증가지역"]}, # '10%초과_20%이하_증가지역', '20%초과_30%이하_증가지역' 은 데이터가 없어서 제외
              hover_data={"category":False, "user_group":False}
              )

fig.update_layout(
    {
        "title": {
            "text": "<b>< [별첨] 월간 지역별 MAU, NRU 추이 비교 ><b>",
            "x": 0.5,
            "y": 0.99,
            "font": {
                "size": 15
            }
        },
        "yaxis1": {
            "title": "NRU(명)",
            "tickformat": ","
        },
        "yaxis5": {
            "title": "MAU(명)",
            "tickformat": ","
        },
        "legend_traceorder": "normal", #normal(default - 데이터 순서대로), reversed(역순)
        "legend_title_text": "거주 지역"
    },
    height = 550
)

fig.update_xaxes(showticklabels=True) # 1행, 2행 모두 x축 thick 출력
fig.update_xaxes(tickformat = '%m월', dtick = "M1", tick0 = "2022-05", title = "")

fig.update_layout(hovermode="x")

fig.show()

### <가입 연도별/월별 NAU>

In [54]:
# SQL로 처리한 csv 불러오기
yearlyAU = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/yearlyAU.csv', encoding = 'utf-8')

## 가입 연도별 월간 활성 유저수(AU) 시각화
fig = px.bar(yearlyAU, x = 'login_year_month', y = 'user_cnt', color = 'create_year', text = 'user_cnt')

fig.update_traces(texttemplate='%{text:,}명', textposition='inside')

fig.update_layout(
    {
        "title": {
            "text": "<b>< 가입 연도별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  },
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "가입 연도",
        "template":'plotly_white',
    }
)

fig.add_vrect(x0="2022-07-17",
              x1="2022-10-16",
              annotation_text="* 2022년 가입유저 기준, 유입보다 이탈 속도가 빠른 구간",
              annotation_position="top left",
              fillcolor="hotpink", opacity=0.25, line_width=0)

fig.add_vrect(x0="2022-10-18",
              x1="2022-11-15",
              annotation_text="* 신규 매장 오픈",
              annotation_position="top left",
              fillcolor="skyblue", opacity=0.25, line_width=0)

fig.show()

In [56]:
## 가입 연월별 활성 유저수(AU) 불러오기
monthlyAU = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/monthlyAu.csv', encoding = 'utf-8')

#시각화
fig = px.bar(monthlyAU,
              x="login_year_month",
              y="user_cnt",
              color="create_year_month",
              text="user_cnt",
             )

fig.update_traces(texttemplate='%{text:,}명', textposition='inside')

fig.update_layout(
    {
        "title": {
            "text": "<b>< 가입 연월별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  },
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "가입 연월",
        "template":'plotly_white',
        "height": 500
    }
)

fig.show()

In [66]:
## 가입 연월별 최초 유입시점 볼륨 대비 미접속률 (어느정도의 사람들이 유입 후 미접속)
AU_ChurnedRate = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/AuChurnedRate.csv', encoding = 'utf-8')

# 데이터 전처리
auChurnedCntAndRate = AU_ChurnedRate.rename(columns = {'first_user_cnt':'M+0_user_count'})
## << auChurnedCntAndRate: 가입 연월별 - 월간(접속 유저수, 재접속률) - M+0부터 추적 가능한 2022-05 유입 유저부터 잘라냄 >>
auChurnedCntAndRate = AU_ChurnedRate[AU_ChurnedRate['create_year_month'] >= '2022-05']\
.sort_values(by=["create_year_month", "login_year_month"])\
.melt(id_vars=["create_year_month", "login_year_month", "create_year", "time_order"], value_vars=["user_cnt", "rate_of_change"])

In [68]:
## <가입연월별 최초유입시점대비 재접속률 변화 시각화

fig = px.line(auChurnedCntAndRate,
              x="time_order",
              y="value",
              color="create_year_month",
              facet_col = "variable",
              facet_col_spacing = 0.05,
              markers=True,
              category_orders={"create_year_month": ["2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12",
                                                     "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12"]},
              hover_data = {"variable":False, "time_order": False} # 마우스 오버시 노출하고 싶지 않은 데이터 제외 처리
              )

fig.update_layout(
    {
        "title": {
            "text": "<b>< [별첨] 가입 연월별 AU의 M+n 재접속률 ><b>",
            "x": 0.5,
            "y": 0.99,
            "font": {
                "size": 13
            }
        },
        "xaxis": {
            "title": "",
            "tick0": "1",
        },
        "xaxis2": {
            "title": "",
            "tick0": "1",
        },
        "legend_traceorder": "normal", #normal(default - 데이터 순서대로), reversed(역순)
        "legend_title_text": "가입 연월"
    },
    height = 350
)

fig.add_vrect(x0="M+0",
              x1="M+2",
              annotation_text="",
              annotation_position="top left",
              fillcolor="hotpink", opacity=0.15, line_width=0)

fig.update_yaxes(title='활성 유저수(명)', row=1, col=1)
fig.update_yaxes(tickformat = ',', row=1, col=1)
fig.update_yaxes(title='재접속률(%)', row=1, col=2)
fig.update_yaxes(tickformat = '.0%', row=1, col=2)
fig.update_yaxes(matches=None, showticklabels=True) # matches=None (y축을 독립적으로 설정)

fig.update_layout(hovermode="x")

fig.show()

### <가입 월별 지역별 M+1 재접속률>

In [71]:
AuChurnedRateByRegion = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/AuChurnedRateByRegion.csv', encoding = 'utf-8')

AuChurnedRateByRegion = AuChurnedRateByRegion.rename(columns = {'first_user_cnt':'M+0_user_count'})
AuChurnedRateByRegion

,login_year_month,create_year_month,create_year,time_order,user_cnt,M+0_user_count,rate_of_change
0,2022-05,2021-07,2021,M0,2,2,1.0
1,2022-05,2021-07,2021,M0,2,2,1.0
2,2022-05,2021-07,2021,M0,135,135,1.0
3,2022-05,2021-07,2021,M0,24,24,1.0
4,2022-06,2021-07,2021,M4,2,2,1.0
...,...,...,...,...,...,...,...
672,2022-11,2022-11,2022,M0,382,382,1.0
673,2022-11,2022-11,2022,M0,36,36,1.0
674,2022-11,2022-11,2022,M0,17,17,1.0
675,2022-11,2022-11,2022,M0,23,23,1.0


## **[잔존율 분석]**

In [76]:
# 유저타입별 월간 일평균 D+day 잔존율
monthly_ret_by_Usertype = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/monthly_retention_by_usertype.csv', encoding = 'utf-8')
# 유저타입별 일간 D+day 잔존율
dauNday_ret = pd.read_csv('/content/drive/MyDrive/Project_1/user_analyze/dauNdayRet.csv', encoding = 'utf-8')

In [79]:
## 신규유저 월간 일평균 N-day 잔존율

#전체 유저의 잔존율 데이터만 가져오기
newMauNdayRet = monthly_ret_by_Usertype[monthly_ret_by_Usertype['user_type'] == '신규유저'] # (전체/신규유저/재방문유저)
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig1 = px.line(newMauNdayRet,
              x="login_year_month",
              y=['d1_ret', 'd7_ret', 'd14_ret', 'd30_ret'],
              line_group="variable",
              text="value",
              markers=True,
             )

fig1.update_traces(
    yaxis="y2",
    textposition='top center',
    texttemplate='%{text: .1%}',
    hovertemplate = '%{y:.1%}',
)

# fig2(mau 그래프) 생성
fig2 = px.bar(newMauNdayRet,
             x="login_year_month",
             y="mau",
            )
fig2.update_traces(marker_color='lightgray')

subfig.add_traces(fig1.data + fig2.data)
subfig.update_layout(
    {
        "title": {
            "text": "<b>< 신규 유저 월간 일평균 N-day 잔존율 ><b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 13
            }
        },
        "xaxis": {
            "title": "",
            "tick0": "2022-05-01",
            "dtick": 'M1',
            "tickformat": "%m월"
        },
        "yaxis": { "title": "MAU(명)", "tickformat": ',' },
        "yaxis2": { "title": "Avg. N-day Retention(%)", "tickformat": '.1%', "range": [0, 0.8]},
        "legend_title_text": "N-day 잔존율",
        "template":'plotly_white',
    }
)

subfig.update_layout(hovermode="x")

# secondary_y 축의 gird 만 보이게 하기
subfig['layout']['yaxis1']['showgrid'] = False

subfig.show()

In [75]:
# step.1 - 전체 유저의 잔존율 데이터만 가져오기
newDauNdayRet = dauNday_ret[dauNday_ret['user_type'] == '신규유저'] # (전체/신규유저/재방문유저)

# step.2 - 서브플랏 할당
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# fig1(잔존율 그래프) 생성
fig1 = px.line(newDauNdayRet,
              x="login_date",
              y=['d1_ret', 'd7_ret', 'd14_ret', 'd30_ret'],
              color="variable",
             )
fig1.update_traces(yaxis="y2") # fig1의 y축을 보조축으로 변환

# fig2(dau 그래프) 생성
fig2 = px.bar(newDauNdayRet,
             x="login_date",
             y="dau",
            )
fig2.update_traces(marker_color='lightgray')

# subfig에 fig1, fig2를 통합 할당
subfig.add_traces(fig1.data + fig2.data)
subfig.update_layout(
    {
        "title": {
            "text": "<b>< 신규 유저 N-day 잔존율 ><b>",
            "x": 0.5,
            "y": 0.8,
            "font": {
                "size": 13
            }
        },
        "xaxis1": {
            "title": "",
            "tick0": "2022-05-01",
            "dtick": 'M1',
            "tickformat": "%Y-%m-%d"
        },
        "yaxis": { "title": "DAU(명)", "tickformat": ',', "range": [0, 1000] },
        "yaxis2": { "title": "N-day Retention(%)", "tickformat": '.0%', "range": [0, 1]},
        "legend_title_text": "N-day 잔존율",
        "template":'plotly_white',
    }
)

# 잔존율 탐색 기준선 도식
subfig.add_hline(y=0.7,
                 line_dash="dot",
                 secondary_y=True,
                 )
# 잔존율 탐색 기준선 관련 내용 표기
subfig.add_annotation(
    x='2022-11-07',
    y=700,
    xref="x",
    yref="y",
    text="잔존율 70% 기준선",
    showarrow=True,
    font=dict(
        size=12,
        color="Gray"
    ),
    align="right",
    arrowhead=2,
    ax=-60,
    ay=-30
)


# 신규 매장 입점 기간 표시
subfig.add_vrect(x0="2022-06-13",
                 x1="2022-06-16",
                 annotation_text="* 신규 매장 입점(6/14 ~ 6/15)",
                 annotation_position="top left",
                 fillcolor="skyblue", opacity=0.25, line_width=0)

subfig.add_vrect(x0="2022-11-06",
                 x1="2022-11-09",
                 annotation_text="* 신규 매장 입점(11/7 ~ 11/8)",
                 annotation_position="top right",
                 fillcolor="skyblue", opacity=0.25, line_width=0)

subfig.add_vrect(x0="2022-11-21",
                 x1="2022-11-30",
                 annotation_text="* 11월 3주차 ~",
                 annotation_position="top right",
                 fillcolor="hotpink", opacity=0.25, line_width=0)

# secondary_y 축의 gird 만 보이게 하기
subfig['layout']['yaxis1']['showgrid'] = False

subfig.show()